In [ ]:
import re
import os
import pandas as pd
import numpy as np

data_dir = os. getcwd()+'/data/'
res_dir = os. getcwd()+'/res/'
header = ['ep1', 'ep2', 'rg', 'se','z', 'p', 'h2_obs', 'h2_obs_se'  'h2_int', 'h2_int_se', 'gcov_int', 'gcov_int_se']

In [ ]:
# Choose the summary stats by the effect size. The bigger, the better.
# formula: ((4 * n_cases * n_controls)/(n_cases + n_controls))

ep = 'G6_MS'
# 'T1D_STRICT', 'K11_COELIAC', 'SLE_FG', 'M13_RHEUMA', 'T2D', 'L12_VITILIGO'
# 'G6_MYASTHENIA', 'J10_ASTHMA', 'G6_MS', 'L12_PSORIASIS'

file_in = data_dir+ep+'.txt' # tsv txt
if not os.path.isfile(file_in):
    file_in = data_dir+ep+'.tsv'
if not os.path.isfile(file_in):
    file_in = data_dir+ep+'.csv'
if not os.path.isfile(file_in):
    print('Data cannot be found!')
file_out = data_dir+ep+'.premunge.gz'
df = pd.read_csv(file_in, sep='\t')
if len(df.columns) == 1:
    df = pd.read_csv(file_in, sep=' ')
if len(df.columns) == 1:
    df = pd.read_csv(file_in)
#     try:
#         df = pd.read_csv(file_in, sep=' ')
#     except pd.errors.ParserError:
#         df = pd.read_csv(file_in)
df = df.rename(columns={
    'rsid': 'snpid', 'SNPID': 'snpid', 'variant_id': 'snpid', 'SNP': 'snpid',
    'effect_allele': 'a1', 'A1': 'a1', 'A1_effect': 'a1',
    'other_allele': 'a2', 'A2': 'a2', 'A2_other': 'a2',
    'p': 'pval', 'P-val': 'pval', 'p_value': 'pval', 'P_EUR': 'pval',
    'OR(A1)': 'or', 'odds_ratio': 'or',
    'standard_error': 'se', 'se_EUR': 'se',
    'beta_EUR': 'beta'
})
print(len(df))
df.head(3)

In [ ]:
# 'K11_IBD', 'K11_CROHN', 'M13_SJOGREN' finn, 'M13_SYSTSLCE' 格式, 'G6_MS'

ep = 'K11_CROHN'
file_in = data_dir+ep+'.txt'
file_out = data_dir+ep+'.premunge.gz'
matrix = []
with open(file_in) as f:
    for line in f:
        row = line.split(' ')
        if len(row) != 8:
            row = row[:7] + [row[-1]]
        matrix.append(row)
matrix = np.array(matrix)
df = pd.DataFrame(matrix[1:], columns=matrix[0])
df['snpid'] = df['SNP\n'].str[:-1]
df = df.rename(columns={
    'Allele1': 'a1',
    'Allele2': 'a2',
    'P.value': 'pval',
    'Effect': 'beta'
})
print(len(df))
df.head(3)

### to download data from Finngen + UKBB:
gsutil cp gs://finngen-production-library-green/finngen_R9/finngen_R9_analysis_data/ukbb_meta/meta/AUTOIMMUNE_HYPERTHYROIDISM_meta_out.tsv.gz /Users/feiwang/Documents/Materials
### to download data from Finngen:
gsutil cp gs://finngen-production-library-green/finngen_R9/finngen_R9_analysis_data/summary_stats/release/finngen_R9_D3_AIHA_OTHER.gz /Users/feiwang/Documents/Materials

In [ ]:
# D3_AIHA_OTHER, D3_ALLERGPURPURA, D3_ITP, E4_ADDISON, G6_DISSOTH, G6_GUILBAR,
# H7_IRIDOCYC_ANTER, L12_DERMATHERP, M13_MCTD, M13_WEGENER

ep = 'M13_WEGENER'
file_in = data_dir+ep+'.gz'
file_out = data_dir+ep+'.premunge.gz'
df = pd.read_csv(file_in, sep='\t')
df = df.rename(columns={
    'rsids': 'snpid',
    'ref': 'a1',
    'alt': 'a2',
})
print(len(df))
df.head(3)

In [ ]:
# L12_ALOPECAREATA, N14_IGA_NEPHROPATHY, GEST_DIABETES, G6_OTHDEMYEL, E4_THYROIDITAUTOIM,
# E4_HYTHY_AI_STRICT, D3_ANAEMIA_B12_DEF, CHIRBIL_PRIM, AUTOIMMUNE_HYPERTHYROIDISM

ep = 'AUTOIMMUNE_HYPERTHYROIDISM'
file_in = data_dir+ep+'.tsv.gz'
file_out = data_dir+ep+'.premunge.gz'
df = pd.read_csv(file_in, sep='\t')
df = df.rename(columns={
    'rsid': 'snpid',
    'REF': 'a1',
    'ALT': 'a2',
    'all_inv_var_meta_p': 'pval',
    'all_inv_var_meta_beta': 'beta'
})
print(len(df))
df.head(3)

In [ ]:
# if the data above looks all good, save it as a zipped tsv
df = df[['snpid', 'a2', 'a1', 'pval', 'beta']]
df.to_csv(file_out, sep='\t', compression='gzip', index=None)

In [ ]:
ep1=T1D_STRICT
ep2=K11_IBD
python munge_sumstats.py \
    --sumstats data/$ep2.premunge.gz \
    --N-cas 2051 \
    --N-con 594747 \
    --out data/$ep2 \
    --merge-alleles w_hm3.snplist \
    --chunksize 500000
python ldsc.py \
    --rg data/$ep1.sumstats.gz,data/$ep2.sumstats.gz \
    --ref-ld-chr eur_w_ld_chr/ \
    --w-ld-chr eur_w_ld_chr/ \
    --out res/$ep1.$ep2

### In terminal, run code below:
<code>

    cd Projects/familial_trajectory/ldsc
    source activate ldsc
    ep=K11_IBD
    python munge_sumstats.py \
        --sumstats data/$ep.premunge.gz \
        --N-cas 3440 \
        --N-con 37031 \
        --out data/$ep \
        --merge-alleles w_hm3.snplist \
        --chunksize 500000

    ep1=T1D_STRICT
    ep2=K11_COELIAC
    python ldsc.py \
        --rg data/$ep1.sumstats.gz,data/$ep2.sumstats.gz \
        --ref-ld-chr eur_w_ld_chr/ \
        --w-ld-chr eur_w_ld_chr/ \
        --out res/$ep1.$ep2
</code>

In [ ]:
def findStats(string):
    stats1 = re.findall(r'data/([\w_\d]+)\.sumstats\.gz', string)
    stats2 = re.findall(r'\s+(\d\.\d+(e\-\d+){0,1})', string)
    stats2 = [float("{:.4f}".format(float(i[0]))) for i in stats2]
    stat_dict = dict(zip(header, stats1+stats2))
    return pd.Series(stat_dict)

def readLog(log_file, df):
    with open(log_file) as f:
        log = f.readlines()
    if len(log) != 0:
#         print(log)
        for line in log:
            if re.match(r'^data/', line):
                df = df.append(findStats(line), ignore_index=True)
    return df

In [ ]:
ep1 = 'T1D_STRICT'
ep2_list = ['K11_COELIAC', 'SLE_FG', 'M13_RHEUMA', 'T2D', 'L12_VITILIGO',
 'G6_MYASTHENIA', 'J10_ASTHMA', 'G6_MS', 'L12_PSORIASIS']
#['K11_COELIAC', 'K11_CROHN', 'K11_IBD', 'SLE_FG', 'L12_VITILIGO', 'M13_RHEUMA', 'T2D']
stat_df = pd.DataFrame(columns=header, dtype=object)
for ep2 in eps:
    f_name = res_dir+ep1+'.'+ep2+'.log'
    if os.path.isfile(f_name):
        stat_df = readLog(f_name, stat_df)
    else:
        print(ep2)
stat_df

In [ ]:
eps = ['T1D_STRICT',
 'M13_RHEUMA',
 'M13_SJOGREN',
 'M13_SYSTSLCE',
#  'M13_DERMATOPOLY',
 'M13_WEGENER',
 'D3_ALLERGPURPURA',
 'M13_MCTD',
 'SLE_FG',
#  'I9_RHEUFEV',
 'G6_MS',
 'G6_DISSOTH',
 'AUTOIMMUNE_HYPERTHYROIDISM',
 'E4_THYROIDITAUTOIM',
 'E4_HYTHY_AI_STRICT',
#  'E4_GRAVES_OPHT_STRICT',
 'E4_ADDISON',
 'D3_AIHA_OTHER',
 'D3_ITP',
 'D3_ANAEMIA_B12_DEF',
 'K11_COELIAC',
 'K11_IBD',
 'G6_MYASTHENIA',
 'G6_OTHDEMYEL',
 'G6_MYOMUSCINOTH',
 'G6_GUILBAR',
 'H7_IRIDOCYC_ANTER',
 'CHIRBIL_PRIM',
 'L12_PSORIASIS',
 'L12_VITILIGO',
 'L12_ALOPECAREATA',
#  'L12_PEMPHIGOID',
 'L12_DERMATHERP',
 'N14_IGA_NEPHROPATHY',
 'T2D',
 'GEST_DIABETES',
 'K11_CROHN',
 'J10_ASTHMA']

In [47]:
pd.read_csv(data_dir+'/autoimmune_gwas_sumstats/GCST90014023_buildGRCh38.tsv', nrows=10, sep='\t')

,variant_id,p_value,chromosome,base_pair_location,effect_allele,other_allele,effect_allele_frequency,beta,standard_error,sample_size
0,rs367896724,0.284,1,10177,AC,A,0.398000,0.059058,0.055112,363495
1,rs555500075,0.731,1,10352,TA,T,0.393000,0.019496,0.056730,363495
2,rs534229142,0.844,1,10511,A,G,0.001310,-0.148768,0.756536,363495
3,rs376342519,0.504,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,0.994000,0.242325,0.362319,363495
4,rs558604819,0.499,1,10642,A,G,0.000214,-1.330179,1.968856,363495
5,rs575272151,0.413,1,11008,G,C,0.086100,-0.077214,0.094349,363495
6,rs544419019,0.413,1,11012,G,C,0.086100,-0.077214,0.094349,363495
7,rs561109771,0.376,1,11063,G,T,0.000046,11.653539,13.165550,363495
8,rs540538026,0.216,1,13110,A,G,0.059300,0.154051,0.124597,363495
9,rs62635286,0.229,1,13116,G,T,0.189000,0.089233,0.074200,363495
